# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. 

Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with 

`mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
dbname ='uk_food'
dblist = mongo.list_database_names()
if dbname in dblist:
    print(f"The database '{dbname}' exists.")
else:
    print(f"The database '{dbname}' does not exist.")

The database 'uk_food' exists.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collections = db.list_collection_names()

In [6]:
# review the collections in our new database
print(f"Collections in {db.name}: {', '.join(collections)}")
collection = db["establishments"]

Collections in uk_food: establishments


In [7]:
# review a document in the establishments collection
doc = collection.find_one()
pprint(doc)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64f2cc181cedc756e05f0858'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
        },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
        },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
# Insert the new restaurant into the collection
result = collection.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print(f"Successfully inserted new restaurant. ID: {result.inserted_id}")
else:
    print("Failed to insert new restaurant.")

Successfully inserted new restaurant. ID: 64f2cc425aa8431f9f82333c


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
doc = collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1}  
)
print(doc)

business_type_id = doc['BusinessTypeID']
business_type_id

{'_id': ObjectId('64f2cc181cedc756e05f0858'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


1

3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
result = collection.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": business_type_id}}
)

In [14]:
# Confirm that the new restaurant was updated
if result.acknowledged:
    print(f"Updated restaurant: Matched {result.matched_count} docs, Modified {result.modified_count} docs")
else:
    print("Failed to update restaurant.")

Updated restaurant: Matched 1 docs, Modified 1 docs


In [15]:
# Show the updated entry to verify
updated = collection.find_one({"BusinessName": "Penang Flavours"})
pprint(updated)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64f2cc425aa8431f9f82333c'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Initial count of Dover establishments: {dover_count}")

Initial count of Dover establishments: 994


In [17]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dovers = collection.delete_many({"LocalAuthorityName": "Dover"})

In [18]:
# Check if any remaining documents include Dover
final_dover_count = collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Final count of Dover establishments: {final_dover_count}")

Final count of Dover establishments: 0


In [19]:
# Check that other documents remain with 'find_one'
doc = collection.find_one({},["BusinessName", "LocalAuthorityName"])
pprint(doc)

{'BusinessName': 'The Pavilion',
 'LocalAuthorityName': 'Folkestone and Hythe',
 '_id': ObjectId('64f2cc181cedc756e05f0b3b')}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {"geocode.latitude": {'$toDouble': "$geocode.latitude"},
                                          "geocode.longitude": {'$toDouble': "$geocode.longitude"}
                                         } 
                                }]
                          )

Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [22]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

In [23]:
# Check that the coordinates and rating value are now numbers
establishments.find_one({},
    ["BusinessName", "geocode.latitude", "geocode.longitude", "RatingValue"])

{'_id': ObjectId('64f2cc181cedc756e05f0b3b'),
 'BusinessName': 'The Pavilion',
 'RatingValue': 5,
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}

In [24]:
 # (Note that the numeric values above do NOT have quotes around them... they are no longer strings.)